In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')


In [ ]:
df["WEEKDAY_APPR_PROCESS_START"].value_counts()

In [ ]:
152107 / 255010

In [ ]:
162993 / 240631

In [ ]:
118319 / 164751

In [ ]:
df.groupby(["WEEKDAY_APPR_PROCESS_START","NAME_CONTRACT_STATUS"]).agg("count")

In [ ]:
a = ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
df["NAME_TYPE_SUITE"] = df["NAME_TYPE_SUITE"].replace(a, 'Accompanied')

In [ ]:
b = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure', 
         'Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness', 'Additional Service', 
         'Education', 'Weapon', 'Insurance', 'House Construction', 'Animals'] 
df["NAME_GOODS_CATEGORY"] = df["NAME_GOODS_CATEGORY"].replace(b, 'others')

In [ ]:
c = ['AP+ (Cash loan)', 'Channel of corporate sales', 'Car dealer']
df["CHANNEL_TYPE"] = df["CHANNEL_TYPE"].replace(c, 'Other_Channel')

In [ ]:
d = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism']
df["NAME_SELLER_INDUSTRY"] = df["NAME_SELLER_INDUSTRY"].replace(d, 'Others')

In [ ]:
e = ['Refusal to name the goal', 'Money for a third person', 'Buying a garage','Gasification / water supply',
     'Hobby','Business development','Buying a holiday home / land','Furniture','Car repairs',
     'Buying a home','Wedding / gift / holiday']
df["NAME_CASH_LOAN_PURPOSE"] = df["NAME_CASH_LOAN_PURPOSE"].replace(e, 'Other_Loan')

In [ ]:
df['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
df['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
df['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
df['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
df['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

In [ ]:
df.drop("NFLAG_LAST_APPL_IN_DAY",axis=1) 

# 2 class var biri sadece 5900 satır.

# New Features

In [ ]:
df['NEW_APP_CREDIT_RATE'] = df['AMT_APPLICATION'] / df['AMT_CREDIT']

# (başvurulan kredi miktarı / bankadaki kredi limiti)

In [ ]:
df["NEW_APP_CREDIT_RATE_RATIO"] = df["NEW_APP_CREDIT_RATE"].apply(lambda x: 1 if(x<=1) else 0)

# x <= 1 ise istediği krediyi almış veya daha fazlasını almış.

In [ ]:
df['NEW_AMT_PAYMENT_RATE'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

# (Bankanın verdiği kredi / aylık taksit tutarı)

In [ ]:
df['NEW_APP_GOODS_RATE'] = df['AMT_APPLICATION'] / df['AMT_GOODS_PRICE']

# (başvurduğu kredi miktarı / almak istediği ürünün fiyatı )

In [ ]:
df['NEW_CREDIT_GOODS_RATE'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

# (Bankanın verdiği kredi miktarı / almak istediği ürünün fiyatı)

In [ ]:
df['NEW_RETURN_DAY'] =  df['DAYS_DECISION'] + df['CNT_PAYMENT'] * 30

# kredinin çıktığı gün (- cinsinden) + kredi taksit sayısı * 30 (yani kredinin gün sayısı)

"""
-180 + 12*30 = -180 +360 = 180
+ ise kredi bitmemiş
-180 + 5*30 = -180 +150 = -30
- ise kredi bitmiş
"""


In [ ]:
df['NEW_DAYS_TERMINATION_DIFF'] = df['DAYS_TERMINATION'] - df['NEW_RETURN_DAY']

# beklenen bitiş günü - kalan gün veya biteli kaç gün olduğu

In [ ]:
df['NEW_DAYS_DUE_DIFF'] =  df['DAYS_LAST_DUE_1ST_VERSION'] - df['DAYS_FIRST_DUE'] 

# (Mevcut başvurunun başvuru tarihine göre, bir önceki başvurunun ilk vadesi ne zamandı?) - (Mevcut başvurunun başvuru tarihine göre, bir önceki başvurunun ilk vadesi ne zaman olmalıydı?)

In [ ]:
df["NEW_CNT_PAYMENT"]=pd.cut(x=df['CNT_PAYMENT'], bins=[0, 12, 60,120], labels=["Kısa", "Orta", "Uzun"])

# kredilerin taksit sayılarını kısa orta uzun diye sınıflandırmak 0-12 kısa | 12-60 orta | 60-120 uzun

In [ ]:
df["NEW_END_DIFF"] = df["DAYS_TERMINATION"] - df["DAYS_LAST_DUE"]

# (Mevcut başvurunun başvuru tarihine göre, bir önceki başvurunun beklenen bitişi ne zamandı? -  Mevcut başvurunun başvuru tarihine göre, bir önceki başvurunun son vade tarihi ne zamandı?)

In [ ]:
weekend = ["SATURDAY","SUNDAY"]                              

df["WEEKDAY_APPR_PROCESS_START"] = df["WEEKDAY_APPR_PROCESS_START"].apply(lambda x : "WEEKEND" if (x in weekend) else "WEEKDAY")

# müşteri başvurusunu hangi gün yapmış. Gün isimleri yerine hafta içi hafta sonu yazıyor.
# hafta içi 58-60% arası çıkıyor. Ctesi = 67% pazar = 71%

In [ ]:
df.head()

In [ ]:
# yeni oluşturulan değişkenlerden sonra, kendisinden oluşturulan değişkenleri droplamalı mıyız?